In [183]:
import os
import getpass
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, ConversationChain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import PromptTemplate
import keys

OPENAI_API_KEY = keys.OPENAI_API_KEY
PINECONE_API_KEY = keys.PINECONE_API_KEY
PINECONE_API_ENV = keys.PINECONE_API_ENV

#there are many options from the above that can be used. Langchain offeres many libraries that allow users to interact with OpenAIs LLMS given that they have an API key 
#for this code however, the lines from embeddings, chat_models, memory, and qa chain are used
# embeddings are used to convert the textual infroamtion in documents into vector data that the LLM can understand and query using the semantics in a users query to find relevant results
# ChatOpenAI gives us the ability to choose which LLM we can use. The LLM will look into the vector databse and contruct an answer. We will use our fine tuned instance once we find a way to make it accessible throught this code
# ChatMessage History allows use to build a history object where the LLM keeps track of user questions and AI responses to keep the conversation relevant
# load_qa_chain is the question an answer chain that allows us to run the a user quer with the vector database. usually this function returns a generic response. but the code below has an input documents similarity parameter to get relevant information



# always remove the API key before committing code to github. 


embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# temperature in the function below is 0. this is the randomness variable. a temperature value > 1 will be completely random. 
# the value below is at 0 to have no randomness. this coupled with our relevant document search is our method of mitigating hallucinations from the LLM

#llm = ChatOpenAI(model='gpt-3.5-turbo',temperature=0, openai_api_key=API_Key)

prompt_template = """Use the following pieces of retrieved documents to answer the question at the end. Please think rationally, take chat history into consideration, and answer from your own knowledge base. If you really can't construct an answer, then answer with 'no'.
{context}
{chat_history}
question: {question}
Answer:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question", "chat_history"])
chain_type_kywargs = {"prompt": PROMPT}


model_name = 'gpt-3.5-turbo'
#llm = ChatOpenAI(model=model_name,temperature=0, openai_api_key=OPENAI_API_KEY)
#chain = load_qa_chain(llm, chain_type='stuff')

# initializing history to track the conversation

history = ChatMessageHistory()


# the value below only shows the messages from the entire conversation history
history.messages

query = ''

global memory

model = ChatOpenAI(model_name=model_name, temperature=0, openai_api_key=OPENAI_API_KEY)
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)


In [184]:
# our database value is vectordb. we are using the file db2 which has our pdfs and CSVs. we had db and db (csv) as test cases aginst information in pdfs and CSVs. 
# we did not know how a vector databse would behave if we gave put differently structued data in one place. but its working out well so far. 

vectordb = Chroma(persist_directory='db2', embedding_function=embeddings)
qa = ConversationalRetrievalChain.from_llm(llm=model, retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={'K': 10}), memory=memory, combine_docs_chain_kwargs={"prompt":PROMPT})
gpt_qa = ConversationChain(llm=model)

In [185]:
# assigning a retriever function to a value. this allows the program to retrieve documents for our similartiy search which matches the semantics of our query to info in the databse

#from langchain.retrievers import SVMRetriever
#retriever = vectordb.as_retriever()

#svm_retriever = SVMRetriever.

In [186]:
# the below is our current chat backend. in later sprints we will have user interface. but the below is fine for our testing purposes
# if the user types exit, the loop ends
while True:
    query = str(input("Human: "))
    if query == 'exit':
        break
    print(f'Human: {query}')
    print()
    response = qa.run(query)
    
    if response == 'no' or response == 'No' or response == 'no.' or response == 'No.':
        print("referring to chatgpt")
        gptresponse = gpt_qa.run(query)
        print(gptresponse)
        memory.save_context({"input": query}, {"output": gptresponse})
        print()
    else:
        print(f'Chat CSEC: {response}')
        print()

Human: what is malware

Chat CSEC: Malware is a general term for all types of malicious software that is used with the aim of attempting to breach a computer system's security policy with respect to confidentiality, integrity, or availability. It refers to a broad class of attacks that are loaded on a system, typically without the knowledge of the legitimate owner, to compromise the system to the benefit of an adversary. Some examples of malware include viruses, worms, Trojan horses, spyware, and bot executables. Malware can infect systems in various ways, such as propagation from infected machines, tricking users to open tainted files, or enticing users to visit malware propagating websites. It can be inserted at any point in the system life cycle and can affect a range of victims, from end user systems to servers.

Human: explain it in more detail



Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Chat CSEC: Malware, short for malicious software, is a term used to describe any type of software that is designed to harm or exploit computer systems. It encompasses a broad range of attacks and can take various forms, including viruses, worms, Trojan horses, spyware, and bot executables.

Viruses are pieces of code that can attach themselves to other system programs and replicate themselves, spreading from one system to another. They can cause damage to files, slow down computer performance, and even render systems inoperable.

Worms are similar to viruses but do not require a host program to spread. They can replicate themselves and spread across networks, often exploiting vulnerabilities in computer systems.

Trojan horses are programs that appear harmless or useful but contain hidden malicious code. They can be used to gain unauthorized access to systems, steal sensitive information, or perform other malicious activities.

Spyware is designed to secretly gather information about a